In [1]:
import turicreate as tc

# Tuote arvostelu data

In [2]:
product = tc.SFrame('amazon_data')

# Datan tutkiminen

In [3]:
product.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


# Sana määrä vector

In [4]:
product['word_count'] = tc.text_analytics.count_words(product['review'])

In [5]:
product.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'the':1.0, 'than': 1.0, 'fu ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


In [6]:
product['name'].show()

Materializing SArray

# Tutkitaan yleisintä tuotetta

In [7]:
most_frequent = product[product['name'] == 'Vulli Sophie the Giraffe Teether'] 

In [8]:
len(most_frequent)

785

In [9]:
most_frequent['rating'].show()

Materializing SArray

# Rakennetaan classifier

In [10]:
product['rating'].show()

Materializing SArray

## Negatiivinen vai positiivinen

In [11]:
product = product[product['rating'] != 3]

In [12]:
product['sentiment'] = product['rating'] >= 4

In [13]:
product

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'the':1.0, 'than': 1.0, 'fu ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


## classifier

In [14]:
train_data, test_data = product.random_split(.8, seed=0)

In [15]:
sentiment_model = tc.logistic_classifier.create(train_data, target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 1.617404     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 1.706276     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 1.833503     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 2.347436     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Mallin arvioiminen

In [16]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   0.001   | 0.698948948948949  | 0.9933514440949385 | 27976 | 5328 |
 |   0.002   | 0.6619744744744744 | 0.9922076065198742 | 27976 | 5328 |
 |   0.003   | 0.6385135135135135 | 0.9909922790963683 | 27976 | 5328 |
 |   0.004   |  0.62256006006006  | 0.9903846153846154 | 27976 | 5328 |
 |   0.005   | 0.6112987987987988 | 0.989848441521304  | 27976 | 5328 |
 |   0.006   | 0.6002252252252253 | 0.9894195024306548 | 27976 | 5328 |
 |   0.007   | 0.589527027027027  | 0.9888475836431226 | 27976 | 5328 |
 |   0.008   | 0.5816441441441441 | 0.9883828996282528 | 27976 | 5328 |
 |   0.009   | 0.574512012012012

# Tuoteen arvostelun ymmärtäminen mallia käyttäen 

In [17]:
most_frequent['predicted_sentiment'] = sentiment_model.predict(most_frequent, output_type='probability')

In [18]:
most_frequent.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'purchase': 1.0,'teething': 1.0, ...",0.9993655365682511
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'a': 1.0, 'is': 1.0,'when': 1.0, 'him': 1.0, ...",0.9998633791689665
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'made': 1.0, 'of': 1.0,'packaging': 1.0, 'no': ...",0.25452681975365593
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'another': 1.0, 'out':1.0, 'run': 1.0, 'lost': ...",0.9165688083898587
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'disappointed': 1.0,'will': 1.0, 'take': ...",0.6855768205793703
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'late': 1.0, 'perfect':1.0, 'pack': 1.0, 'on ...",0.99999994452112
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'delight': 1.0, 'in':1.0, 'other': 1.0, ...",0.9979351181092867
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'inside': 1.0, 'water':1.0, 'don': 1.0, 'up': ...",0.9999745004834408
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'its': 1.0, 'fine': 1.0,'is': 1.0, 'which': 1.0, ...",0.946014442833353
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'off': 1.0, 'have': 2.0,'of': 1.0, 'some': 1.0, ...",0.383011361414439


## Arvosteluiden järjesteleminen sentimentin mukaan

In [19]:
most_frequent = most_frequent.sort('predicted_sentiment', ascending=False)

In [20]:
most_frequent.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'around': 1.0,'explore': 1.0, 'they': ...",1.0
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'will': 1.0, '15months':1.0, 'would': 2.0, ...",1.0
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'11': 1.0, 'prisrob':1.0, '12': 1.0, 'who': ...",1.0
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'out': 1.0, 've': 1.0,'would': 1.0, 'enough': ...",0.9999999999998377
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5.0,"{'whether': 1.0,'neutral': 1.0, 'gend ...",0.9999999999998284
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5.0,"{'penny': 1.0, 'little':1.0, 'perfect': 1.0, ...",0.9999999999997958
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4.0,"{'worth': 1.0, 'works':1.0, 'teether': 1.0, ...",0.9999999999994914
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5.0,"{'question': 1.0, 'must':1.0, 'overall': 1.0, ...",0.9999999999941254
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'makers': 1.0, 'or':1.0, 'take': 1.0, 'can': ...",0.999999999987423
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4.0,"{'dye': 1.0, 'of': 1.0,'cause': 1.0, 'fade': ...",0.9999999999829475


In [21]:
most_frequent[0]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

In [22]:
most_frequent[1]['review']

"As a mother of 16month old twins; I bought Sophie [1 for each, of course] when they were 4 months old after careful reading of all reviews. I heard great things about Sophie and wanted to give her a try. At 4 months babies can't do much more than grasp and semi gnaw on Sophie. For many months I had to squeeze Sophie myself [which I personally enjoyed] and set it on their laps. They LOVED Sophie. The squeak is LOUD and sounds exactly like a dog's squeaky chew toy, just for those who are wondering.As they grew and their motor skills developed to each milestone, Sophie gained more and more individual babytime. The twins were able to squeeze her themselves and chew on her around the clock. They love to throw her, stretch her, squeeze her, chew on her, drool on her... you name it, they have done it. One of the two Sophie's took an extended vacation out in the back yard [unbeknownst to me] and once found, a little water had her looking like a champ again... ready to face another day of play

In [23]:
most_frequent[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [24]:
most_frequent[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 

# Tehtävät

In [25]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [26]:
 words = product['word_count'][1]['the']
words

1.0

In [27]:
'family' in product['word_count'][0]


False

In [28]:
def awesome_count(word, selecteds):
    if selecteds in word:
        return word[selecteds]
    else:
        return 0

In [29]:
awesome_count(product['word_count'], 'awesome')

0

In [30]:
for i in selected_words:
    product[i] = product['word_count'].apply(lambda x: awesome_count(x, i))

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [31]:
product = product.sort('great', ascending=False)

In [32]:
product

name,review,rating,word_count,sentiment,awesome
"Baby Jogger 2011 CityMini Double Stroller, ...","Before purchasing thisstroller, I read several ...",5.0,"{'survive': 1.0,'drying': 1.0, 'troub ...",1,1.0
"Dream On Me 3&quot;Playard Mattress, White ...","Mattress fit perfectly.Measures 26x38. Great, ...",5.0,"{'to': 1.0, 'forward':1.0, 'looking': 1.0, ...",1,0.0
"Britax 2012 B-AgileStroller, Red ...",[I got this stroller formy daughter prior to the ...,4.0,"{'around': 1.0, 'bill':1.0, 'umbrella': 1.0, ...",1,0.0
Summer Infant Wide ViewDigital Color Video ...,I love this baby monitor.I can compare this on ...,5.0,"{'happy': 1.0,'extremely': 1.0, ...",1,0.0
Ubbi Cloth Diaper PailLiner ...,"(updated 3.22.13) Afterextensive research, t ...",5.0,"{'thanks': 1.0, 'wade':1.0, 'trying': 1.0, ...",1,0.0
Medela Harmony ManualBreast Pump ...,I have to say anythingMedela is great. Always ...,5.0,"{'in': 1.0, 'worth': 1.0,'price': 1.0, 'prepar ...",1,0.0
Summer Infant Day andNight Handheld Color ...,We use this monitorconstantly and we love ...,5.0,"{'like': 1.0, 'who': 1.0,'moms': 1.0, ...",1,0.0
"UPPAbaby G-Luxe Stroller,Jake/Black ...",UPDATE: 2.5 years in andwe still love it. We ...,5.0,"{'been': 1.0, 'having':1.0, 'problem': 2.0, ...",1,0.0
Alva One Size WashableReusable Minky Cloth ...,this diaper has snaps foradjusting that are great ...,5.0,"{'mommy': 1.0, 'price':1.0, 'my': 1.0, 'on': ...",1,0.0
Ergo Baby BC2EPK1 GalaxyGrey Baby Carrier with ...,I purchased this carrierfor an amazing price! ...,5.0,"{'if': 1.0, 'strap': 1.0,'are': 1.0, 'they': 1.0, ...",1,0.0


In [33]:
train_data, test_data = product.random_split(.8, seed=0)

In [34]:
selected_words_model = tc.logistic_classifier.create(train_data, target='sentiment', features=selected_words)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 126775

Number of classes           : 2

...

| 4         | 5        | 0.078354     | 0.847367          | 0.855537            |

| 5         | 6        | 0.097693     | 0.847359          | 0.855537            |

| 6         | 7        | 0.116432     | 0.847359          | 0.855537            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [35]:
selected_words_model.coefficients.print_rows(num_rows=12)

+-------------+-------+-------+----------------------+----------------------+
|     name    | index | class |        value         |        stderr        |
+-------------+-------+-------+----------------------+----------------------+
| (intercept) |  None |   1   |  1.3335354220097368  | 0.009142012941805668 |
|   awesome   |  None |   1   |  1.0498743303992557  |  0.0838739573783708  |
|    great    |  None |   1   |  0.8676645988132632  | 0.019461514266571483 |
|  fantastic  |  None |   1   |  0.9066012481517547  | 0.11696239976325241  |
|   amazing   |  None |   1   |  1.0899576031225766  | 0.10147902931529358  |
|     love    |  None |   1   |  1.3924284679336925  | 0.029184511373079227 |
|   horrible  |  None |   1   | -2.2188045718663036  |  0.083453451728675   |
|     bad     |  None |   1   | -0.9910774446955636  | 0.03912779630024927  |
|   terrible  |  None |   1   |  -2.251234772100727  | 0.08106895084009506  |
|    awful    |  None |   1   | -2.0627298676836894  | 0.1052458

In [36]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8461746336776363,
 'auc': 0.6917727160315987,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  428  |
 |      1       |        0        |  176  |
 |      1       |        1        | 27753 |
 |      0       |        1        |  4947 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9155024823104455,
 'log_loss': 0.398662449953141,
 'precision': 0.8487155963302753,
 'recall': 0.9936983064198504,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27929 | 5375 |

In [37]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9649591640643767,
 'auc': 0.9825649491609536,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  651  |
 |      1       |        0        |  516  |
 |      1       |        1        | 27413 |
 |      0       |        0        |  4724 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9791581090493455,
 'log_loss': 0.11717824883154226,
 'precision': 0.9768030216647663,
 'recall': 0.9815245801854703,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

In [38]:
diaper_champ_reviews = product[product['name'] == 'Baby Trend Diaper Champ']

In [39]:
diaper_champ_reviews

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,"This item is great, youcan you any bags you ...",5.0,"{'price': 1.0, 'genie':1.0, 'for': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1.0, 'how': 1.0,'like': 1.0, 'on': 1.0, ...",1,0.0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1.0, 'in':1.0, 'each': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1.0,'would': 1.0, 'with': ...",1,0.0
Baby Trend Diaper Champ,I was a diaper genie useruntil my son was about 1 ...,5.0,"{'better': 1.0, 'way':1.0, 'this': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I bought this trying tosave a few dollars and ...,5.0,"{'inexpensive': 1.0,'helpful': 1.0, 'an': ...",1,0.0
Baby Trend Diaper Champ,I laughed when my husbandwanted this product. I ...,5.0,"{'things': 1.0, 'week':1.0, 'about': 2.0, 'o ...",1,0.0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{'satisfied': 1.0,'extremely': 1.0, 'm': ...",1,0.0
Baby Trend Diaper Champ,This product has beengreat. It's easy to ...,5.0,"{'full': 1.0, 'pail':1.0, 'that': 1.0, 'dr ...",1,0.0
Baby Trend Diaper Champ,Great diaper pail. It isvery simple. Uses any ...,5.0,"{'recommend': 1.0,'definitely': 1.0, 'in': ...",1,0.0


In [40]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [41]:
diaper_champ_reviews

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,"This item is great, youcan you any bags you ...",5.0,"{'price': 1.0, 'genie':1.0, 'for': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1.0, 'how': 1.0,'like': 1.0, 'on': 1.0, ...",1,0.0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1.0, 'in':1.0, 'each': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1.0,'would': 1.0, 'with': ...",1,0.0
Baby Trend Diaper Champ,I was a diaper genie useruntil my son was about 1 ...,5.0,"{'better': 1.0, 'way':1.0, 'this': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I bought this trying tosave a few dollars and ...,5.0,"{'inexpensive': 1.0,'helpful': 1.0, 'an': ...",1,0.0
Baby Trend Diaper Champ,I laughed when my husbandwanted this product. I ...,5.0,"{'things': 1.0, 'week':1.0, 'about': 2.0, 'o ...",1,0.0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{'satisfied': 1.0,'extremely': 1.0, 'm': ...",1,0.0
Baby Trend Diaper Champ,This product has beengreat. It's easy to ...,5.0,"{'full': 1.0, 'pail':1.0, 'that': 1.0, 'dr ...",1,0.0
Baby Trend Diaper Champ,Great diaper pail. It isvery simple. Uses any ...,5.0,"{'recommend': 1.0,'definitely': 1.0, 'in': ...",1,0.0


In [42]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [43]:
diaper_champ_reviews

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1.0, 'have': 1.0,'pieces': 1.0, 'betwe ...",1,0.0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1.0,'knows': 1.0, 'parent': ...",1,0.0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1.0,'suggestions': 1.0, ...",1,0.0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1.0, 'any':1.0, 't': 1.0, 'isn': ...",1,0.0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'either': 1.0, 'be':1.0, 't': 1.0, 'not': ...",1,0.0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'hope': 1.0, 'make':1.0, 'slower': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1.0, 'product':1.0, 'recommend': 1.0, ...",1,0.0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1.0, 'baby': 1.0,'recommended': 1.0, ' ...",1,0.0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1.0, 'told':1.0, 'all': 1.0, ...",1,0.0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1.0, 'be': 1.0,'you': 2.0, 'sell': 1.0, ...",1,0.0


In [44]:
diaper_champ_reviews[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."

In [45]:
a = selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

In [46]:
a

dtype: float
Rows: 1
[0.7914248337329385]

In [47]:
dict= {}

In [70]:
def most(word, select):
    a = 0
    for i in word:
        if select in word:
            a = a + word[select]
    dict[select] = a

In [71]:
for i in selected_words:
    most(product[0]['word_count'], i)

In [72]:
dict

{'awesome': 673.0,
 'great': 6730.0,
 'fantastic': 0,
 'amazing': 1346.0,
 'love': 0,
 'horrible': 0,
 'bad': 673.0,
 'terrible': 0,
 'awful': 0,
 'wow': 0,
 'hate': 0}